In [6]:
import math
import geohash

def wetBuldCalculator(temp_celsius, rh):
    return temp_celsius \
    * math.atan(0.151977 * (rh + 8.313659)**(1/2)) \
    + math.atan(temp_celsius + rh) \
    - math.atan(rh - 1.676331) \
    + 0.00391838 * (rh)**(3/2) \
    * math.atan(0.023101 * rh) \
    - 4.686035

def confortIndex(dry_buld, rh):
    dry_buld_c = dry_buld - 273.15
    wet_buld_c = wetBuldCalculator(dry_buld_c, rh)
    ci = 15 + 0.4 * ((dry_buld_c + wet_buld_c) * (9.0 / 5.0) + 32)
    return ci



45.17689245771212

In [7]:
#geohash.encode(42.392469, -71.215895)

'drt35mq4j2gz'

In [15]:
import datetime

def parseLine(line):
    variables = line.split("\t")
    try:
        milliseconds = int(variables[0])
        dt = datetime.datetime.fromtimestamp(milliseconds/1000.0)
        lat = float(variables[1])
        lon = float(variables[2])
        rh = float(variables[8])
        temperatureK = float(variables[10])
        ci = confortIndex(temperatureK, rh)
        gh = geohash.encode(lat, lon)
        return (gh, dt.month, ci)
    except:
        return ('', 0, 0)
    
text_file = sc.textFile("hdfs://orion11:12001/pj3/3hr_sample/sampled_2015/")


def month_ci_location(gh):
    parsed_data = text_file \
    .map(lambda line: parseLine(line)) \
    .filter(lambda data: data[0].startswith(gh)) 
    month_ci_goruped = parsed_data.map(lambda data: (data[1], data[2])).groupByKey()
    result = []
    for element in month_ci_goruped.collect():
        count = 0
        ci_sum = 0
        for ci in element[1]:
            count += 1
            ci_sum += ci
        result.append((element[0], ci_sum/count))
    return result


In [17]:
boston = month_ci_location("drt")
print(boston)


[(1, 21.23956149906619), (2, 16.75861740434057), (3, 26.080226764546047), (4, 35.49179530520997), (5, 46.7949357763197), (6, 47.81048121769007), (7, 53.41606557872664), (8, 53.37288359330368), (9, 49.891941460302334), (10, 40.59872148461839), (11, 35.54453028416755), (12, 31.971730938078675)]
